In [ ]:
from __future__ import print_function


import os
import glob
import numpy as np
from skimage.transform import resize
from skimage.io import imsave
from PIL import Image
from skimage.io import imread
from skimage.transform import resize
from skimage.io import imsave
import matplotlib as mp
from matplotlib.colors import Normalize
import cv2
name='val'
 
data_path = ''

image_rows = int(512)
image_cols = int(512)
image_depth = 16
if(name=='train'):
    img_input=os.path.join(data_path, 'Tac/')
if(name=='mask'):
    img_input=os.path.join(data_path, 'BMask/')
if(name=='test'):
    img_input=os.path.join(data_path, 'Test/')
if(name=='val'):
    img_input=os.path.join(data_path, 'Val/')

In [ ]:
tot=0
images = sorted(os.listdir(img_input))
for image_name in images:
    print(image_name)
    img = imread(os.path.join(img_input, image_name), as_grey=True)
    tot+=len(img)
    print(img.shape)
tot_batch=tot//image_depth
print('Total Batch')
print(str(tot_batch))

if(name=='mask'or (name=='val')):
    imgs = np.ndarray((tot_batch, image_depth, image_rows, image_cols), dtype=np.uint8)#Mask
    imgs_mask = np.ndarray((tot_batch, image_depth, image_rows, image_cols), dtype=np.uint8)   
if((name=='train')or(name=='test')):
    imgs = np.ndarray((tot_batch, image_depth, image_rows, image_cols), dtype=np.float32)#Immagini
    imgs_mask = np.ndarray((tot_batch, image_depth, image_rows, image_cols), dtype=np.float32) 


In [ ]:
i=0
for image_name in images:
    k=0
    if (i==tot_batch-1):
        break
    print(image_name)
    path=os.path.join(img_input, image_name)
    img = imread(path, as_grey=True)
    if((name=='train')or(name=='test')):
        img[np.where(img<-150)]=-150#IMG
        img[np.where(img>2300)]=2300#IMG
        img = cv2.normalize(img, None, norm_type=cv2.NORM_MINMAX)
        print(img.shape)
    if(name=='mask'or(name=='val')):
        img=resize(img, (len(img),512,512),anti_aliasing=False) #per le mask
        img[np.where(img>0)]=1  
        print(img.shape)
    for j in range(0 , len(img)):
        #print(k)#Debug
        if(k+image_depth>len(img)):
            break
        if(k+image_depth<=len(img)):
            for z in range (0, image_depth):
                if((name=='train')or(name=='test')):
                    imgs[i,z]=(img[k,:,:])
                if(name=='mask'or(name=='val')):  
                    imgs[i,z]=(img[k,:,:])#Per Mask
                #print(imgs[i,z])
                k+=1
                if(z==image_depth-1):
                    i+=1

if(name=='train'):
    np.save('imgs_train.npy', np.expand_dims(imgs,axis=0))
if(name=='mask'):
    np.save('imgs_mask_train.npy', np.expand_dims(imgs,axis=0))
if(name=='test'):
    np.save('imgs_test.npy', np.expand_dims(imgs,axis=0))
if(name=='val'):
    np.save('val_test.npy', np.expand_dims(imgs,axis=0))
print('File Saved')
print(np.expand_dims(imgs,axis=0).shape)
